In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
sys.path.append('/root/code')

from definitions import LOG_DIR, WEIGHT_DIR
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time
import datetime
import numpy
from utils.contrastiveLoss import ContrastiveLoss
from models.SCNN18_Flatten import SCNN18_Flatten
from models.BYOL import BYOL
from models.SCNN18_random_aug import SCNN18_random_aug
import utils.dataset as dataset
import logging
from logging import handlers

In [2]:
LOG = logging.getLogger('root')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('BYOL.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')

In [3]:
initLog()

In [4]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [5]:
# Define model architecture & define hyper-parameter
lr = 1e-5
max_epochs = 50
sample_size=[32000, 1]
batch_size = 64
input_shape = tuple(sample_size)
output_shape = 2

In [6]:
origin_path = ['./SCNN18_0.1second/SCNN-BYOL-Jamendo-FMA-train_30s.h5']
random_augment_path = ['./SCNN18_0.1second/SCNN-BYOL-Jamendo-FMA-train_30s-random_augment.h5']

In [7]:
def generate_batched_data(ds_path):
    for index, ds in enumerate(ds_path):
        if index == 0:
            train_ds = dataset.get_dataset_without_label(ds)
            slices_ds = tf.data.Dataset.from_tensor_slices((train_ds))
        else:
            train_ds = dataset.get_dataset_without_label(ds)
            add_ds = tf.data.Dataset.from_tensor_slices((train_ds))
            slices_ds.concatenate(add_ds)
    dataset_length = [i for i, _ in enumerate(slices_ds)][-1] + 1

    return dataset_length, slices_ds.shuffle(dataset_length, seed=8, reshuffle_each_iteration=True).batch(batch_size)

In [8]:
ds_length, origin_ds = generate_batched_data(origin_path)
ds_length, random_augment_ds = generate_batched_data(random_augment_path)

In [9]:
strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])
with strategy.scope():
    optimizer = get_optimizer('adam', lr)

    # 创建 BYOL 模型
    byol_model = BYOL(input_shape=input_shape)
    byol_model.compile(optimizer=optimizer)

    # 分布式训练循环
    for epoch in range(max_epochs):
        step = 0
        for o_batch, ra_batch in zip(origin_ds, random_augment_ds):
            step += 1
            start_time = time.time()

            inputs_1 = tf.convert_to_tensor(o_batch, dtype=tf.float32)
            inputs_2 = tf.convert_to_tensor(ra_batch, dtype=tf.float32)

            def distributed_train_step(inputs_1, inputs_2):
                with tf.GradientTape() as tape:
                    loss = byol_model(inputs_1, inputs_2)
                grads = tape.gradient(loss, byol_model.trainable_variables)
                byol_model.optimizer.apply_gradients(zip(grads, byol_model.trainable_variables))
                byol_model.update_target_network()
                return loss

            per_replica_losses = strategy.run(distributed_train_step, args=(inputs_1, inputs_2))
            mean_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_losses, axis=None)

            end_time = time.time()

            if step % 10 == 0:
                print(f"Epoch {epoch + 1}, Step {step}/{ds_length // batch_size}, Loss: {mean_loss.numpy()}, taken_time: {end_time - start_time:.3f}")

        LOG.info(f"Epoch {epoch + 1}, Loss: {mean_loss.numpy()}")

    # 保存权重
    byol_model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SCNN_BYOL_Jamendo_FMA_pretrain_30epochs_lr=1e-5.h5"))
    print("Training complete. Weights saved to 'BYOL_SCNN_pretrain.h5'.")

Epoch 1, Step 10/44, Loss: 2.3254237174987793, taken_time: 1.939
Epoch 1, Step 20/44, Loss: 1.5065809488296509, taken_time: 1.911
Epoch 1, Step 30/44, Loss: 1.3159046173095703, taken_time: 1.951
Epoch 1, Step 40/44, Loss: 0.9632480144500732, taken_time: 1.971


2024-07-28 08:58 INFO Epoch 1, Loss: 0.7915165424346924


Epoch 2, Step 10/44, Loss: 0.5248597860336304, taken_time: 1.997
Epoch 2, Step 20/44, Loss: 0.38934096693992615, taken_time: 1.988
Epoch 2, Step 30/44, Loss: 0.25765368342399597, taken_time: 1.977
Epoch 2, Step 40/44, Loss: 0.18287543952465057, taken_time: 2.028


2024-07-28 08:59 INFO Epoch 2, Loss: 0.1660401076078415


Epoch 3, Step 10/44, Loss: 0.13221341371536255, taken_time: 1.997
Epoch 3, Step 20/44, Loss: 0.11530232429504395, taken_time: 2.029
Epoch 3, Step 30/44, Loss: 0.09465350955724716, taken_time: 2.020
Epoch 3, Step 40/44, Loss: 0.08176670223474503, taken_time: 2.012


...

Epoch 29, Step 10/44, Loss: 0.0008081626147031784, taken_time: 2.017
Epoch 29, Step 20/44, Loss: 0.001990118995308876, taken_time: 2.008
Epoch 29, Step 30/44, Loss: 0.0009456612169742584, taken_time: 2.032
Epoch 29, Step 40/44, Loss: 0.001205654814839363, taken_time: 1.987


2024-07-28 09:40 INFO Epoch 29, Loss: 0.0011838740902021527


Epoch 30, Step 10/44, Loss: 0.0027642417699098587, taken_time: 1.988
Epoch 30, Step 20/44, Loss: 0.0015776418149471283, taken_time: 2.023
Epoch 30, Step 30/44, Loss: 0.0010992325842380524, taken_time: 2.007
Epoch 30, Step 40/44, Loss: 0.002368580549955368, taken_time: 1.997


2024-07-28 09:42 INFO Epoch 30, Loss: 0.0021518999710679054


Training complete. Weights saved to 'BYOL_SCNN_pretrain.h5'.
